In [1]:
# URL = 'https://arxiv.org/pdf/2403.18802.pdf'
URL = '/Users/shanekercheval/Downloads/ud923-birrell-nelson-paper.pdf'

# when extracting the text from the PDF, INCLUDE_AT and EXCLUDE_AT are used to determine where to
# start and stop extracting text. For example, if INCLUDE_AT is 'Abstract' and EXCLUDE_AT is
# 'Acknowledgements', then the text extraction will start at (and include) the first occurrence of
# 'Abstract' and stop at (and Exclude) the first occurrence of 'Acknowledgements'.

# INCLUDE_AT = "ABSTRACT"
INCLUDE_AT = None
# EXCLUDE_AT = "Acknowledgements"
EXCLUDE_AT = "REFERENCES"

MODEL = 'gpt-4o-mini'
SYSTEM_MESSAGE = 'You are an AI assistant that gives detailed and intuitive explanations.'
MAX_TOKENS=None
TEMPERATURE=0.8

---

In [2]:
from IPython.display import Markdown, display
from source.library.pdf import clean_text_from_pdf, extract_text_from_pdf
from llm_workflow.openai import OpenAIChat, num_tokens, MODEL_COST_PER_TOKEN


def create_model():
    return OpenAIChat(MODEL, SYSTEM_MESSAGE, max_tokens=MAX_TOKENS, temperature=TEMPERATURE)

In [4]:
# download and extract text of pdf
text = extract_text_from_pdf(pdf_path=URL)
n_tokens = num_tokens(model_name=MODEL, value=text)
print(f"# of tokens: {n_tokens:,}")
print(f"Cost if input tokens == {n_tokens:,}:  ${MODEL_COST_PER_TOKEN[MODEL]['input'] * n_tokens:.3f}")  # noqa: E501
print(f"Cost if output tokens == {n_tokens:,}: ${MODEL_COST_PER_TOKEN[MODEL]['output'] * n_tokens:.3f}")  # noqa: E501


# of tokens: 15,090
Cost if input tokens == 15,090:  $0.002
Cost if output tokens == 15,090: $0.009


In [7]:
# removed text before `INCLUDE_AT` and after `EXCLUDE_AT`
chars_before = len(text)
print(f"{chars_before:,} characters before")
text = clean_text_from_pdf(text=text, include_at=INCLUDE_AT, exclude_at=EXCLUDE_AT)
chars_after = len(text)
print(f"{chars_after:,} characters after")
print(f"Removed {abs((chars_after - chars_before) / chars_before):.2%} of text")
print("Preview:\n---\n")
print(text[:500])
print("\n...\n")
print(text[-500:])

70,292 characters before
67,802 characters after
Removed 3.54% of text
Preview:
---

Implementing Remote Procedure Calls

ANDREW D. BIRRELL and BRUCE JAY NELSON

Xerox Palo Alto Research Center

Remote procedure calls (RPC) appear to be a useful paradig m for providing communication across a  network between programs written in a high-level language. This paper describes a package providing  a remote procedure call facility, the options that face the designer of such a package, and the decisions

~we made. We describe the overall structure of our RPC mechanism, our facilities fo

...

rmance without adopting  extreme measures, and without sacrificing useful call and parameter semantics.

The techniques for managing transport level connections so as to minimize the  communication costs and the state that must be maintained by a server are  important in our experience of servers dealing with large numbers of users. Our  binding semantics are quite powerful, but conceptually simple for a 

In [8]:
n_tokens = num_tokens(model_name=MODEL, value=text)
print(f"# of tokens: {n_tokens:,}")
print(f"Input cost if input tokens == {n_tokens:,}:   ${MODEL_COST_PER_TOKEN[MODEL]['input'] * n_tokens:.3f}")
print(f"Output cost if output tokens == {n_tokens:,}: ${MODEL_COST_PER_TOKEN[MODEL]['output'] * n_tokens:.3f}")
print("\n---\n")

with open("source/library/prompts/summarize_pdf.txt") as f:
    summary_prompt = f.read()

model = create_model()
prompt = f"""
{summary_prompt}

Here is the paper:

{text}
"""

print("# of Tokens and Cost including the prompt")
n_tokens = num_tokens(model_name=MODEL, value=prompt)
print(f"# of tokens: {n_tokens:,}")
print(f"Input cost if input tokens == {n_tokens:,}:   ${MODEL_COST_PER_TOKEN[MODEL]['input'] * n_tokens:.3f}")
print(f"Output cost if output tokens == {n_tokens:,}: ${MODEL_COST_PER_TOKEN[MODEL]['output'] * n_tokens:.3f}")

# of tokens: 14,086
Input cost if input tokens == 14,086:   $0.002
Output cost if output tokens == 14,086: $0.008

---

# of Tokens and Cost including the prompt
# of tokens: 27,254
Input cost if input tokens == 27,254:   $0.004
Output cost if output tokens == 27,254: $0.016


In [9]:

response = model(prompt)
with open('summary.txt', 'w') as f:
    f.write(response)
cost = model.cost
print(f"Cost: ${cost:.3f}")
display(Markdown(response))

Cost: $0.005


# Implementing Remote Procedure Calls

## Introduction

Remote Procedure Calls (RPC) is a communication paradigm designed to facilitate communication across a network between software programs written in a high-level language.

- The fundamental concept behind RPC is to extend the familiar mechanism of **procedure calls** (used for transferring control and data within a single program) to enable communications across a network. 
- When the calling program invokes a remote procedure, it temporarily suspends its operation, sends parameters over the network to the target environment (the callee), executes the procedure there, and returns results to resume operation as if it were a local call.

The motivation for implementing RPC includes:
- **Clean and Simple Semantics**: This makes it easier to construct and debug distributed applications.
- **Efficiency**: Procedure calls can be optimized for rapid communication.
- **Generality**: Procedures are a central feature in algorithms for communication within single-machine computations.

Despite its theoretical appeal, full-scale implementations of RPC have historically been limited. This paper presents the design and implementation of an RPC facility developed for the Cedar project, which provides insights into design choices and performance optimizations.

## Environment

The RPC package was designed primarily for use within the **Cedar programming environment**, which operates across the Xerox research internetwork. 

- Cedar emphasizes powerful and convenient programming, with features supporting highly interactive user interfaces and ease of debugging.
- The environment primarily involves **Dorado** machines, known for their high performance, utilizing a 3-megabit-per-second Ethernet for communication.

The internetwork comprises multiple Ethernets connected by various links, enabling RPC communication to largely occur over local networks, minimizing the impact of lower data rates on users.

## Aims

The main purpose of the RPC project is to simplify the construction of distributed applications, as building communicating programs has traditionally been a complex task that only experts could manage. 

### Primary Goals:
- **Ease of Use**: Enable communication as easily as local procedure calls to encourage experimentation with distributed applications.
- **Efficiency**: Maintain communication efficiency to prevent developers from avoiding network communication due to performance concerns.
- **Powerful Semantics**: Ensure that RPC semantics are robust enough to eliminate the need for additional mechanisms atop the RPC package.
- **Security**: Include provisions for secure communication, addressing a gap in previous protocols.

### Secondary Goals:
- Achieve a communication efficiency within a factor of five of necessary network transmission times.
- Design RPC semantics that retain simplicity without sacrificing power or performance.

## Fundamental Decisions

The design choices for the RPC package include opting for procedure calls over alternatives like message passing or remote forking.

- The decision to use procedure calls stems from their integration into the **Mesa** programming language, which is central to the environment.
- A shared address space among computers was dismissed due to the complexities it would introduce and the inefficiencies in implementation.
- A guiding principle was to keep RPC semantics closely aligned with local procedure calls to enhance usability for programmers experienced in single-machine operations.

### Key Design Principles:
1. **Simplicity**: The RPC calls should mimic local procedure calls to minimize user complexity.
2. **Efficiency**: Aim for high performance while maintaining the ease of use.
3. **Robustness**: Ensure the system can adequately handle communication and machine failures.

## Structure

The RPC system's architecture incorporates several components that facilitate its operations:

- **User**: Initiates the RPC.
- **User-Stub**: Acts as an intermediary that prepares and sends requests.
- **RPCRuntime**: Manages the communication and ensures reliable transmission.
- **Server-Stub**: Receives requests and invokes the appropriate procedure on the server.
- **Server**: Executes the procedure and returns results.

When a user invokes an RPC, the user-stub packages the procedure call and parameters into packets for transmission by the RPCRuntime, which manages retransmission, acknowledgments, and routing.

### Stub Generation:
A tool named **Lupine** generates the user and server stubs based on interface modules defined in the Mesa programming language. 

- The interface module contains procedure names along with argument types, enabling compile-time type checks and appropriate calling sequences without manual coding of communication logic.

## Binding

The binding process involves two key elements: specifying the desired interface and locating the appropriate exporter.

### Naming
- The RPC mechanism binds an importer of an interface to an exporter. Each interface is defined by a type and an instance, which serve to identify the procedures to be invoked remotely.

### Locating an Exporter
The **Grapevine** distributed database is employed for binding in the RPC package. 

- Grapevine maintains records of the network addresses for exported interfaces, permitting reliable and efficient lookups.
- This approach avoids hardcoding addresses in application programs and minimizes interference with other network activity.

### Binding Process:
1. An exporter registers its interface with Grapevine, identifying its export procedure and associated network address.
2. An importer requests binding by querying Grapevine, which returns the address of an active exporter along with its unique identifier.
3. Once bound, the user-stub can make remote calls to the specified procedures seamlessly.

## Packet-Level Transport Protocol

### Requirements
While RPC could utilize existing transport protocols, the authors argue that a specialized protocol tailored for RPC communications yields significant performance advantages.

### Simple Calls
The protocol aims to minimize the real-time elapsed between initiating a call and receiving results. 

- Call packets include a unique call identifier to manage requests and responses, allowing the system to efficiently handle lost packets and duplicate calls.

### Complicated Calls
For calls involving multiple packets, the protocol ensures sequences are acknowledged appropriately, maintaining the integrity of data transfer.

- Mechanisms for retransmission and acknowledgment are built into the protocol for reliability.

### Exception Handling
The RPC implementation mimics the exception handling in the Mesa language by allowing server procedures to signal exceptions back to the caller, maintaining the semantics of local calls.

## Performance

Performance tests conducted on the RPC system show promising results in terms of efficiency and response times for remote calls.

- The measurements take into account various factors, including communication load and network conditions. 
- The results indicate that communication overhead is minimal, particularly for local calls across the same network.

## Status and Discussions

The RPC package has been fully implemented and utilized in various projects including file servers and real-time applications. 

- The performance and usability of the system suggest that RPC can facilitate the development of new distributed applications that were previously infeasible due to communication complexities.
- Ongoing research will address any emerging needs for further optimizations or additional features, particularly regarding security and protocol enhancements.

The design and implementation of the RPC package highlighted here represent a significant advancement in making distributed systems easier to build and more reliable in performance.

In [10]:
print(f"Total Cost:            ${model.cost:.5f}")
print(f"Total Tokens:          {model.total_tokens:,}")
print(f"Total Prompt Tokens:   {model.input_tokens:,}")
print(f"Total Response Tokens: {model.response_tokens:,}")

Total Cost:            $0.00492
Total Tokens:          28,713
Total Prompt Tokens:   27,342
Total Response Tokens: 1,371


---

In [ ]:
prompt = "Do the authors discuss the specifics or different scenarios of when not to use cosine similarity?"
response = model(prompt)
cost = model.cost
print(f"Cost: ${cost:.3f}")
display(Markdown(response))

---